In [ ]:
from bottle import route, run, template, static_file
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm_notebook as tqdm # Progress bar
# tqdm().pandas()
from IPython.display import display, HTML

wejscie = 0
wynik2 = pd.DataFrame()

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')

@route('/')
def glowna():
    return template('index', zawartosc='<strong><u>Strona główna</u></strong><p></p>')

@route("/wykresy")
@route("/wykresy/")
def wykresy():
    return template("chart-chartjs", data_scatter=dane_scatter, data_box=Dict_bar, zawartosc= '<strong><u>Wykresy</u></strong><p></p>')

@route("/wykresy2")
@route("/wykresy2/")
def wykresy2():
    return template("chart-chartjs2", labels_pie=labels_pie, dane_pie=dane_pie, dane_radar=list_radar, zawartosc= '<strong><u>Wykresy2</u></strong><p></p>')

@route('/kod')
@route('/kod/')
def kod():
    return template('kod', zawartosc= '<strong><u>Kod</u></strong><p></p>')

@route('/basic_table')
@route('/basic_table/')
def table():
    merg = pd.read_csv("test500.csv")
    z = HTML(merg.to_html())
    tables=merg.to_html()
    #tables = HTML( merg.to_html().replace("\\n","<br>"))
    return template('table', tabela=tables, zawartosc= '<strong><u>Tabela</u></strong><p></p>')
      
@route('/scrap')
@route('/scrap/')
def scrap():
    global wejscie
    global wynik2
    global dane_scatter
    global Dict_bar
    if (wejscie == 0):
        df_check = wynik2
        wejscie+=1
    else:
        df_check = wynik2
        
###############################
#POCZATEK SCRAPERA
###############################
    def basic_info_list(num_pages):
        dane_ranking=[]    
        for i, x in enumerate(tqdm(range(1, num_pages+1))):             
            bs = BeautifulSoup(requests.get("https://www.filmweb.pl/films/search?orderBy=popularity&descending=true&page="+str(x)).text)
            lista = bs.find('ul', {'id':'searchResult', 'class':['resultsList', 'hits']})
            for j, li in enumerate(lista):
                single=[]
    # TITLE -> JESLI NIE ZAGRANICZNY, TO KRAJOWY
                if (li.find('div', {"class":"filmPreview__originalTitle"}) != None):
                    single.append(li.find('div', {"class":"filmPreview__originalTitle"}).text)
                else: 
                    single.append(li.find('h2', {"class":"filmPreview__title", "itemprop":"name"}).text)
    # MAIN GENRE:     
                single.append(li.find('div', {"class":"filmPreview__info--genres"}).find('a').text)
    # DATE            
                single.append(li.find('div', {"class":"filmPreview__year", "itemprop":"datePublished"})['content'])
    # DIRECTOR:
                single.append(li.find('div', {"class":"filmPreview__info--directors"}).find('span', {"itemprop":"name"}).text)
    # RATING
                single.append(li.find('span', {'class':'rateBox__rate', 'itemprop':'ratingValue'}).text) 
    # NUMBER OF REVIEWS:            
                single.append(li.find('div', {"class":"filmPreview__extends"}).findChildren('span', {"class":["rateBox__votes", "rateBox__votes--count"], "itemprop":"ratingCount"})[0]['content'])
    # MOVIE LINK:
                single.append("https://www.filmweb.pl"+li.find('a', {"class":"filmPreview__link", "itemprop":"url"})['href'])
                dane_ranking.append(single)
        return dane_ranking
    #--------------------------------------------------------------------
    def pre_process_basic(ll):
        df = pd.DataFrame(ll, columns=["Title", "Main genre", "Date", "Director", "Average Rating", "Number of reviews", "Link"])
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        df['Date']= pd.to_datetime(df['Date'])
        df['Number of reviews'] = df['Number of reviews'].astype(int)
        df['Average Rating'] = df['Average Rating'].map(lambda x: x.replace(',','.'))
        return df
    #--------------------------------------------------------------------
    def detailed(basic_scrapped):
        movie_list = basic_scrapped.Link.values
        dane_detail=[]
        for i, x in enumerate(tqdm(movie_list)):
            single=[]
            bs = BeautifulSoup(requests.get(x).text)   
            tabela_header = bs.find('div', {'class':'filmOtherInfoSection__group'}).findAll('div', {'class':'filmInfo__header'})
            tabela_info = bs.find('div', {'class':'filmOtherInfoSection__group'}).findAll('div', {'class':'filmInfo__info'})
            slownik = dict([[o.text,j] for o,j in zip(tabela_header, tabela_info) if o.text in ['boxoffice', 'budżet', 'studio']])                                         
    #         print(slownik)
        # BOX OFFICE:
            if ('boxoffice' in slownik.keys()):
                try:
                    single.append(slownik['boxoffice'].find('div', {'class': [None, 'boxoffice']}).text)
                except:
                    print("ERROR w BOXOFFICE", basic_scrapped.loc[[i]].Title)
            else: 
                single.append(np.nan)
                print("BRAK INFORMACJI O BOXOFFICE", basic_scrapped.loc[[i]].Title, '\n----------------------------------------------')   
        # BUDGET:
            if ('budżet' in slownik.keys()):
                single.append(slownik['budżet'].text)                                                       
            else: 
                single.append(np.nan)
                print("BRAK INFORMACJI O BUDZECIE", basic_scrapped.loc[[i]].Title, '\n----------------------------------------------')
        # PRODUCTION COMPANY:
            if ('studio' in slownik.keys()):
                single.append(slownik['studio'].text)                                                       
            else: 
                single.append(np.nan)
                print("BRAK INFORMACJI O PRODUCENCIE", basic_scrapped.loc[[i]].Title, '\n----------------------------------------------')            
        # AWARDS:
            try:
                single.append(bs.find('a', {'class':'awards__link'}).text)
            except:
                print("BRAK INFORMACJI O NAGRODACH", '\n----------------------------------------------')  
                single.append('0')
            if ((i+1)%30==0 & i+1!=len(movie_list)): 
                print('----------------------------------------------\nPRZERWA ANTYBOTOWA\n----------------------------------------------')
                sleep(1)
            dane_detail.append(single)
        return dane_detail
    #--------------------------------------------------------------------
    def producent(x):
        if (re.search(r'\(', x)): return re.search(r'(.*)(?= \()', x).group(0)
        return x
    #--------------------------------------------------------------------
    def pre_process_detailed(basic_scrapped):
        dane_detailed = pd.DataFrame(detailed(basic_scrapped), columns=['BoxOffice', 'Budget', 'Producer', 'Awards_Nominations'])
        dane_detailed.dropna(inplace=True)
        dane_detailed.BoxOffice = dane_detailed.BoxOffice.map(lambda x: re.search(r'[0-9]+', re.sub(" ", "", x)).group(0), na_action='ignore')    
        dane_detailed.Budget = dane_detailed.Budget.map(lambda x: re.search(r'[0-9]+', re.sub(" ", "", x)).group(0), na_action='ignore')    
        dane_detailed.Producer = dane_detailed.Producer.map(lambda x: re.split(r'  /  ', x), na_action='ignore')
        # MAIN PRODUCER:    
        main_producer = dane_detailed['Producer'].values.tolist()
        dane_detailed['Main Producer'] = dane_detailed['Producer'].map(lambda x: producent(x[0]))
        return dane_detailed
    #--------------------------------------------------------------------
    def award_split(string):
        if (string == '0'): return ['0', '0', '0']
        award_list = re.split(r', | i | oraz | i', string)
        award_output=[]
        for i, x in enumerate(award_list):
    # 3 AWARD/NOMINATIONS TYPES:
            if (len(award_list)==3):
                if (re.search('ę', x) != None): 
                    award_output.append(1)
                    continue
                else: 
                    award_output.append(re.search(r'([0-9]+)', x).group(0))
                    continue
    # 2 AWARD/NOMINATIONS TYPES: 
            elif (len(award_list)==2):
                if (i==1): 
                    award_output.append(0)
                if (re.search('ę', x) != None): 
                    award_output.append(1)
                    continue
                if (re.search('ę', x) == None): 
                    award_output.append(re.search(r'([0-9]+)', x).group(0))
                    continue
    # 1 AWARD/NOMINATION TYPE:     
            elif (len(award_list)==1):
                if (i==0): 
                    award_output.extend(('0', '0')) 
                if (re.search('ę', x) != None): 
                    award_output.append(1)
                    continue
                if (re.search('ę', x) == None): 
                    award_output.append(re.search(r'([0-9]+)', x).group(0))
                    continue
        return award_output
    #--------------------------------------------------------------------
    basic_scrapped = pre_process_basic(basic_info_list(60))
    #--------------------------------------------------------------------
    dane_detailed = pre_process_detailed(basic_scrapped)
    #--------------------------------------------------------------------
    wynik = pd.concat([basic_scrapped, dane_detailed],axis=1)
    wynik.dropna(inplace=True)
    wynik.reset_index(drop=True, inplace=True)
    #--------------------------------------------------------------------
    nagrody = (
        pd.DataFrame(
            np.asarray(
            [award_split(x) for x in wynik['Awards_Nominations'].values.tolist()]
            ),
            columns=['Main Awards', 'Additional Awards', 'Nominations']
        )
    )
    #--------------------------------------------------------------------
    wynik2 = pd.concat([wynik, nagrody],axis=1)
    #--------------------------------------------------------------------
    wynik2.to_csv(r'C:\Users\szymo\Desktop\Nauka\Python\Projekt\Projekt Python\test500.csv')
    
###############################
#KONIEC SCRAPERA
###############################
    
    test500 = pd.read_csv("test500.csv")

    dataset = []

#     myColor = ["#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF","#2CBFCE","#93C667","#52AA5E","#F45B69","#01BAEF"]
#     for i, (name, group) in enumerate(test500.groupby("Main genre")):
#         #print(name)
#         groupDict = {}
#         groupDict["label"] = name
#         datapoints = []
#         for row in group.itertuples():
#             datapoints.append({"x":row.Nominations, "y":row.BoxOffice, "r":row.Budget})
#         groupDict["data"] = datapoints.copy()
#         groupDict["backgroundColor"] = myColor[i]
#         groupDict["hoverBackgroundColor"] = myColor[i]
#         dataset.append(groupDict.copy())
#     global data_filmweb_bubbles
#     data_filmweb_bubbles = json.dumps(dataset)
    
#     merg = pd.read_csv("test500.csv")
#     dane_scatter = merg[['BoxOffice', 'Budget']].rename(columns = {'BoxOffice':'x', 'Budget':'y'})
#     dane_scatter = dane_scatter[["x", "y"]].to_dict('records')
#     dane_scatter = json.dumps(dane_scatter)


    #SCATTER PLOT
    merg = pd.read_csv("test500.csv")
    Dict_scatter={}
    Dict_scatter["label"] = 'Zależność Box Office od Budżetu'
    dane_scatter = merg[['BoxOffice', 'Budget']].rename(columns = {'BoxOffice':'y', 'Budget':'x'})                                   
    Dict_scatter["data"] = dane_scatter[["x", "y"]].to_dict('records')
    Dict_scatter["backgroundColor"] = 'rgb(65, 148, 217)'
    dane_scatter = json.dumps([Dict_scatter])
    
    #BOX PLOT
    dataset = merg[['BoxOffice', 'Main Producer']].groupby('Main Producer').mean()
    dataset.sort_values(by=['BoxOffice'], inplace=True)
    dataset = dataset.tail(20)
    Dict_bar={}
    Dict_bar['type']='bar'
    Dict_bar['x']=dataset.index.values.tolist()
    Dict_bar['y']=dataset.BoxOffice.values.tolist()
    Dict_bar = json.dumps([Dict_bar])

    #PIE PLOT
    global labels_pie
    global dane_pie
    dataset_pie = merg[['Main genre', 'BoxOffice']].groupby('Main genre').sum()
    dataset_pie.sort_values(by=['BoxOffice'], inplace=True)
    dataset_pie = dataset_pie.tail(5)
    dane_pie = json.dumps(dataset_pie.BoxOffice.values.tolist())
    labels_pie = json.dumps(dataset_pie.index.values.tolist())
    # display(dane_pie, labels_pie)
    
    #RADAR
    global list_radar 
    data_radar_1 = merg[['BoxOffice', 'Budget', 'Number of reviews', 'Average Rating', 'Nominations']]
    data_radar_1 -= data_radar_1.min()
    data_radar_1 /= data_radar_1.max()
    data_radar_1 *= 100
    data_radar_1 = pd.concat([data_radar_1, merg['Main genre']], axis= 1)
    data_radar_1 = data_radar_1.loc[data_radar_1['Main genre'].isin(['Dramat', 'Akcja', 'Fantasy'])]
    data_radar_1 = data_radar_1.groupby('Main genre').mean()

    list_radar = json.dumps([{'type': 'scatterpolar', 
    'r': data_radar_1.iloc[0].values.tolist(), 
    'theta': data_radar_1.columns.tolist(),
    'fill': 'toself',
    'name': data_radar_1.index[0]},

    {'type': 'scatterpolar', 
    'r': data_radar_1.iloc[1].values.tolist(), 
    'theta': data_radar_1.columns.tolist(),
    'fill': 'toself',
    'name': data_radar_1.index[1]},

    {'type': 'scatterpolar', 
    'r': data_radar_1.iloc[2].values.tolist(), 
    'theta': data_radar_1.columns.tolist(),
    'fill': 'toself',
    'name': data_radar_1.index[2]}])

    #SPRAWDZENIE
    if (df_check.equals(wynik2)):
        return template('scrap_porazka', zawartosc= '<strong><u>Scrap porazka</u></strong><p></p>')
    else:
        return template('scrap_sukces', zawartosc= '<strong><u>Scrap sukces</u></strong><p></p>')
        
run(host='localhost', port=8001, debug=True)

Bottle v0.12.18 server starting up (using WSGIRefServer())...
Listening on http://localhost:8001/
Hit Ctrl-C to quit.

127.0.0.1 - - [27/Jan/2021 11:06:17] "GET / HTTP/1.1" 200 52466
C:\Users\szymo\anaconda3\envs\myPython\lib\site-packages\ipykernel_launcher.py:66: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/60 [00:00<?, ?it/s]

C:\Users\szymo\anaconda3\envs\myPython\lib\site-packages\ipykernel_launcher.py:103: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/600 [00:00<?, ?it/s]

BRAK INFORMACJI O BUDZECIE 3    Léon
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 14    Intouchables
Name: Title, dtype: object 
----------------------------------------------
----------------------------------------------
PRZERWA ANTYBOTOWA
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 30    Seksmisja
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 30    Seksmisja
Name: Title, dtype: object 
----------------------------------------------
ERROR w BOXOFFICE 32    Dzień świra
Name: Title, dtype: object
BRAK INFORMACJI O BUDZECIE 32    Dzień świra
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 42    Scent of a Woman
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 59    La vita è bella
Name: Title, dtype: object 
--------------------------------

...

  0%|          | 0/600 [00:00<?, ?it/s]

BRAK INFORMACJI O BUDZECIE 3    Léon
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 14    Intouchables
Name: Title, dtype: object 
----------------------------------------------
----------------------------------------------
PRZERWA ANTYBOTOWA
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 30    Seksmisja
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 30    Seksmisja
Name: Title, dtype: object 
----------------------------------------------
ERROR w BOXOFFICE 32    Dzień świra
Name: Title, dtype: object
BRAK INFORMACJI O BUDZECIE 32    Dzień świra
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 42    Scent of a Woman
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 59    La vita è bella
Name: Title, dtype: object 
--------------------------------

BRAK INFORMACJI O BOXOFFICE 304    Sami swoi
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 304    Sami swoi
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 316    Death Proof
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 322    Vinci
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 322    Vinci
Name: Title, dtype: object 
----------------------------------------------
----------------------------------------------
PRZERWA ANTYBOTOWA
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 336    Pokłosie
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 336    Pokłosie
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 338    Wesele
Name: Title, dtyp

----------------------------------------------
PRZERWA ANTYBOTOWA
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 483    Pitbull. Niebezpieczne kobiety
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 483    Pitbull. Niebezpieczne kobiety
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BOXOFFICE 484    Cube
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 484    Cube
Name: Title, dtype: object 
----------------------------------------------
ERROR w BOXOFFICE 495    Najlepszy
Name: Title, dtype: object
BRAK INFORMACJI O BUDZECIE 495    Najlepszy
Name: Title, dtype: object 
----------------------------------------------
BRAK INFORMACJI O BUDZECIE 508    Loving Vincent
Name: Title, dtype: object 
----------------------------------------------
----------------------------------------------
PRZERWA ANTYBOTOWA
------

C:\Users\szymo\anaconda3\envs\myPython\lib\site-packages\bottle.py:195: DeprecationWarning: Template encodings other than utf8 are no longer supported.
  value = obj.__dict__[self.func.__name__] = self.func(obj)
127.0.0.1 - - [27/Jan/2021 11:18:45] "GET /scrap/ HTTP/1.1" 200 8236
